In [1]:
import torch
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS

import pandas as pd
from matplotlib import pyplot as plt
import random
import time
import models
import gdown

from scipy.stats import nbinom,norm,truncnorm
from scipy.integrate import trapezoid as trapz

In [2]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTltw5xQVB_sSBCLiA5nzbiDc1srkInw5TDBWcYy50A-Yhr7zKMTgJUy0aoE1q0uCo5WUJSJSVhR_SY/pub?gid=0&single=true&output=csv"
df = pd.read_csv(url)

In [3]:
#file_path = 'results.ods'

# Read the Excel file
#df = pd.read_excel(file_path, engine="odf")

# Print the DataFrame
print(df)

winners=np.array(df['Winner'])
losers=np.array(df['Loser'])
loser_scores=np.array(df['Points'])
names = pd.concat([df['Winner'], df['Loser']]).unique()
ids = list(range(len(names)))
dict_to_id = dict(zip(names,ids))
dict_to_name=dict(zip(ids,names))
winner_ids = np.array([dict_to_id[i] for i in winners])
loser_ids = np.array([dict_to_id[i] for i in losers])

   Winner   Loser  Points
0    Fedo   Mauro       1
1    Fedo   Idris       3
2   Mauro   Idris       6
3    Fedo   Idris       5
4   Mauro   Idris       8
5    Fedo   Mauro       5
6   Mauro   Frago       7
7   Mauro   Frago       7
8   Frago   Mauro       6
9   Frago   Idris       9
10  Frago   Idris       9
11  Frago   Idris      10
12  Frago   Idris       8
13  Mauro  Satvik       7
14   Fedo   Idris       4
15   Fedo   Idris       9


In [435]:
winner_ids=torch.Tensor(winner_ids).int()
loser_ids=torch.Tensor(loser_ids).int()
loser_scores=torch.Tensor(loser_scores).int()

In [436]:
# Run inference using NUTS (No-U-Turn Sampler) in MCMC
nuts_kernel = NUTS(models.pyro_model)
mcmc_run = MCMC(nuts_kernel, num_samples=100, warmup_steps=10)
mcmc_run.run(winner_ids, loser_ids, loser_scores)

# Get posterior samples
posterior_samples = mcmc_run.get_samples()




Sample: 100%|██| 110/110 [00:39,  2.79it/s, step size=5.15e-03, acc. prob=0.519]


In [437]:
# Print the posterior mean and standard deviation of player strengths
for i in range(len(ids)):
    player = f'Player {i+1}: {dict_to_name[i]}'
    print(player)
    print("Posterior mean of strength:", posterior_samples['strength'][:, i].mean().item())
    print("Posterior std dev of strength:", posterior_samples['strength'][:, i].std().item())
    print()

Player 1: Fedo
Posterior mean of strength: -1.5651113986968994
Posterior std dev of strength: 0.7456702589988708

Player 2: Mauro
Posterior mean of strength: 1.172225832939148
Posterior std dev of strength: 0.579407274723053

Player 3: Frago
Posterior mean of strength: 1.2006733417510986
Posterior std dev of strength: 0.6641607284545898

Player 4: Idris
Posterior mean of strength: -0.7139475345611572
Posterior std dev of strength: 0.3505500555038452

Player 5: Satvik
Posterior mean of strength: 0.8312307596206665
Posterior std dev of strength: 0.48502063751220703



## Adding Gibbs sampling (or something similar)

In [6]:
if isinstance(winner_ids, torch.Tensor):
    winner_ids=winner_ids.numpy()
    loser_ids=loser_ids.numpy()
    loser_scores=loser_scores.numpy()

### Choice: option 1

In [4]:
l1=0.5
epsilon=1e-6
std=0.2

def unnorm_product_function(l1,lambdas_wins, lambdas_losses, scores, std=0.2, epsilon=1e-6):
    if l1<0 or l1>1:
        return 0
    else:
        #p=np.concatenate(((l1+epsilon)/(lambdas_wins+l1+epsilon),(lambdas_losses+epsilon)/(lambdas_losses+l1+epsilon)))
        p=np.concatenate(((l1+epsilon)/(lambdas_losses+l1+epsilon),(lambdas_wins+epsilon)/(lambdas_wins+l1+epsilon)))
        func=np.prod(nbinom.pmf(11,scores,p))*norm.pdf(l1,0.5,std)
        return func


def slice_sampler(x_init=l1,custom_function=unnorm_product_function, n_samples=10):
    x = x_init
    samples = np.zeros(n_samples)
    for i in range(n_samples):
        # Draw a vertical line
        y = np.random.uniform(0, custom_function(x))
        # Create a horizontal “slice” (i.e., an interval)
        x_left = x - 0.1
        while y < custom_function(x_left):
            x_left -= 0.1
        x_right = x + 0.1
        while y < custom_function(x_right):
            x_right += 0.1
        # Draw new sample
        if x_left<0:
            x_left=0.01
        if x_right>1:
            x_right=0.99
        #print(x_left,x_right)
        while True:
            x_new = np.random.uniform(x_left, x_right)
            if y < custom_function(x_new):
                break
            elif x_new > x:
                x_right = x_new
            elif x_new < x:
                x_left = x_new
            else:
                raise Exception("Slice sampler shrank to zero!")
        x = x_new
        samples[i] = x
    return samples[-1]

def init_strengths(n,std):
    mean = 0.5
    # replace with your value
    lower, upper = 0, 1
    # Because truncnorm takes its bounds in the standard Gaussian space, 
    # you should convert your bounds
    lower_std, upper_std = (lower - mean) / std, (upper - mean) / std
    # Create the truncated Gaussian distribution
    truncated_gaussian = truncnorm(lower_std, upper_std, loc=mean, scale=std)
    # Sample from the distribution
    sample = truncated_gaussian.rvs(n)
    return sample

def simulate_match(p):
    count_heads = np.random.binomial(11, p)
    count_tails = 11-count_heads
    while count_heads < 11 and count_tails < 11:
        # draw a sample from a Bernoulli distribution
        sample = np.random.binomial(1, p)
        if sample == 1:
            count_heads += 1
        else:
            count_tails += 1
    if count_heads ==11:
        return 1
    else:
        return 0

class gibbs_model:
    def __init__(self, winner_ids, loser_ids, loser_scores,prior_std=0.2):
        #find the number of players
        self.n_players=len(np.unique(np.concatenate((winner_ids,loser_ids))))
        #initialize the strengths; you can also put a wider prior
        self.strengths=init_strengths(self.n_players,prior_std)
        self.winner_ids=winner_ids
        self.loser_ids=loser_ids
        self.loser_scores=loser_scores
        self.posterior_samples=None
        
    def posterior_sampling(self,n_iterations=100, warmup=20):
        strengths_time=[]
        ids = list(range(self.n_players))
        thermalized=False
        for i in range(n_iterations):
            print("\r" + "Progress: [" + "#" * i + " " * (n_iterations - i) + f"] {int(100 * i / n_iterations)}%", end="")
            if thermalized==False:
                if i==warmup:
                    thermalized=True
            random.shuffle(ids)
            for current_id in ids:
                #print(current_id)
                l1=self.strengths[current_id]
                #print(l1)
                #print(np.where(self.winner_ids == current_id))
                lambdas_wins=self.strengths[self.loser_ids[np.where(self.winner_ids == current_id)]]
                scores=self.loser_scores[np.where(self.winner_ids == current_id)]
                lambdas_losses=self.strengths[self.winner_ids[np.where(self.loser_ids == current_id)]]
                scores=np.concatenate((scores,self.loser_scores[np.where(self.loser_ids == current_id)]))
                custom_function = lambda l1: unnorm_product_function(l1, lambdas_wins, lambdas_losses, scores, std, epsilon)
                self.strengths[current_id] = slice_sampler(x_init=l1, custom_function=custom_function)
                #self.strengths[current_id]=slice_sampler()
            if thermalized:
                strengths_time.append(self.strengths.copy())
        self.posterior_samples=np.array(strengths_time)
        
    def predict_match(self, player_1_id, player_2_id, n_matches=1000, explicit=False):
        player_1_wins=0
        l1=np.random.choice(self.posterior_samples[:,player_1_id], size=n_matches, replace=True, p=None)
        l2=np.random.choice(self.posterior_samples[:,player_2_id], size=n_matches, replace=True, p=None)
        for match in range(n_matches):
            player_1_wins+=simulate_match(l1[match]/(l1[match]+l2[match]))
        ratio=player_1_wins/n_matches
        if explicit:
            print(f'Player 1 has {ratio*100}% probability of winning')
        return ratio
    
    def print_table(self):
        # Print the posterior mean and standard deviation of player strengths
        mean=np.mean(self.posterior_samples,axis=0)
        std=np.std(self.posterior_samples,axis=0)
        for i in range(len(ids)):
            player = f'Player {i+1}: {dict_to_name[i]}'
            print(player)
            print("Posterior mean of strength:", mean[i])
            print("Posterior std dev of strength:", std[i])
            print()
            
        for i in ids:
            for j in ids[i+1:]:
                print(f'{dict_to_name[i]} has {self.predict_match(i,j)}% of probability of winning against {dict_to_name[j]}')

In [5]:
g=gibbs_model(winner_ids, loser_ids, loser_scores)

In [6]:
g.posterior_sampling(n_iterations=500, warmup=100)

Progress: [################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################### ] 99%

In [10]:
g.print_table()

Player 1: Fedo
Posterior mean of strength: 0.8320186215077634
Posterior std dev of strength: 0.10251444627445608

Player 2: Mauro
Posterior mean of strength: 0.7129986140566582
Posterior std dev of strength: 0.11379314742392181

Player 3: Frago
Posterior mean of strength: 0.6142742026033647
Posterior std dev of strength: 0.10826966948272507

Player 4: Idris
Posterior mean of strength: 0.47552939379559483
Posterior std dev of strength: 0.08508386347745425

Player 5: Satvik
Posterior mean of strength: 0.5178458582935604
Posterior std dev of strength: 0.15310752724200838

Fedo has 0.651% of probability of winning against Mauro
Fedo has 0.735% of probability of winning against Frago
Fedo has 0.872% of probability of winning against Idris
Fedo has 0.854% of probability of winning against Satvik
Mauro has 0.607% of probability of winning against Frago
Mauro has 0.804% of probability of winning against Idris
Mauro has 0.744% of probability of winning against Satvik
Frago has 0.696% of probabi